## Логистическая регрессия

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from torchvision.transforms import v2
from PIL import Image, ImageOps

import albumentations as A

**Предобработка изображений**

In [ ]:
from sklearn.decomposition import PCA

def load_images(data='train', pca_components=100):
    path = f"dataset/{data}/"
    list_cat = os.listdir(path)
    
    X, y = ([], [])
    
    transform = A.Compose([
    A.Rotate(limit=180, p=1.0),
    A.GaussNoise(p=0.5), 
    A.RandomBrightnessContrast(p=0.25),
    ])

    for i, cat in enumerate(list_cat):        
        list_images = os.listdir(path + cat)
        
        for j in list_images:          
            with Image.open(path + cat + "/" + j) as img:
                # приводим изображения к чёрно-белому формату и одному размеру
                img = np.array(v2.Resize(size=(256, 256))(ImageOps.grayscale(img)))
                
                for p in range(5): 
                    # применяем аугментации 4 раза 
                    if p:
                        img = transform(image=img)["image"]
                    
                    # применяем PCA с pca_components главными компонентами
                    pca = PCA(pca_components)
                    img_pca = pca.fit_transform(img)
                
                    # вытягиваем в одномерный массив и добавляем в матрицу объекты-признаки
                    X.append(img_pca.flatten())
                    y.append(i)
    
    return np.array(X), np.array(y)


X_train, y_train = load_images(data='train', pca_components=75)
X_test, y_test = load_images(data='test', pca_components=75)

**Обучение модели**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score

# модель
model = LogisticRegression(tol=1e-2, max_iter=1000, random_state=123)
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

# результаты 
print(f'accuracy: {accuracy_score(y_test, y_pred_test):.4f}')